<a href="https://colab.research.google.com/github/aya1101/NER-resume/blob/main/NER_resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dataturks_resume_entities_for_ner_path = kagglehub.dataset_download('dataturks/resume-entities-for-ner')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import pandas as pd


In [ ]:
data = pd.read_json("/kaggle/input/resume-entities-for-ner/Entity Recognition in Resumes.json", lines=True)
data['content'] = data['content'].str.replace('\n',' ')
data['content'] = data['content'].str.replace('\n',' ')
data = data.drop(['extras'], axis=1)
data.head()

In [ ]:
data.iloc[0]['annotation']

In [ ]:
def extract_information(df_existing):
    entity_list = []

    for index, row in df_existing.iterrows():
        annotation_data = row["annotation"]

        # Nếu annotation_data không phải danh sách, đưa nó về danh sách
        if not isinstance(annotation_data, list):
            annotation_data = [annotation_data]

        for annotation in annotation_data:
            label = annotation["label"][0] if annotation["label"] else ""

            for point in annotation.get("points", []):
                text = point.get("text", "").strip()
                start = point.get("start", 0)

                # Tách nội dung theo dấu "•" và tính toán vị trí start-end cho từng phần
                sub_texts = [sub.strip() for sub in text.split("•") if sub.strip()]
                current_start = start

                for sub in sub_texts:
                    current_end = current_start + len(sub)
                    entity_list.append((current_start, current_end, label))
                    current_start = current_end + 1  # Giả định khoảng cách 1 ký tự giữa các mục

    # Thêm danh sách entity vào dataframe ban đầu như một cột mới
    df_existing["entity_list"] = [entity_list] * len(df_existing)
    return df_existing

In [ ]:
data = extract_information(data)

In [ ]:
data.head(10)

In [ ]:
data.info()

In [ ]:
data

# LSTM

Step 1: Tokenize data

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
label2id = {
    "O": 0, "B-Skills": 1, "I-Skills": 2, "B-College Name": 3, "I-College Name": 4,
    "B-Graduation Year": 5, "I-Graduation Year": 6, "B-Designation": 7, "I-Designation": 8,
    "B-Companies worked at": 9, "I-Companies worked at": 10, "B-Email Address": 11,
    "I-Email Address": 12, "B-Location": 13, "I-Location": 14, "B-Name": 15, "I-Name": 16,
    "B-Degree": 17, "I-Degree": 18, "B-Years of Experience": 19, "I-Years of Experience": 20
}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
def align_labels_with_tokens(text, entity_list):
    tokenized = tokenizer(text, return_offsets_mapping=True, truncation=True, padding="max_length", max_length=128)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]
    offset_mapping = tokenized["offset_mapping"]

    # Khởi tạo tất cả là 'O'
    labels = ["O"] * len(input_ids)

    # Gán nhãn dựa trên vị trí start-end trong entity_list
    for start, end, label_type in entity_list:
        first = True

        for i, (token_start, token_end) in enumerate(offset_mapping):
            # Bỏ qua [CLS], [SEP], [PAD] (token có offset_mapping là (0, 0))
            if token_start == 0 and token_end == 0:
                continue

            if token_start >= start and token_end <= end:
                labels[i] = f"I-{label_type}" if not first else f"B-{label_type}"
                first = False

    # Chuyển labels sang ID
    label_ids = [label2id.get(label, 0) for label in labels]
    while len(label_ids) < 128:
        label_ids.append(label2id["O"])
    return input_ids, attention_mask, label_ids


In [ ]:
def tokenize_data(data):
    all_input_ids, all_attention_masks, all_labels = [], [], []

    for _, row in data.iterrows():
        text = row["content"]
        if isinstance(row["entity_list"], list):
            entity_list = row["entity_list"]
        else:
            entity_list = ast.literal_eval(row["entity_list"])  # Chuyển JSON thành list

        input_ids, attention_mask, label_ids = align_labels_with_tokens(text, entity_list)

        all_input_ids.append(input_ids)
        all_attention_masks.append(attention_mask)
        all_labels.append(label_ids)

    return all_input_ids, all_attention_masks, all_labels


In [ ]:
all_input_ids, all_attention_masks, all_labels = tokenize_data(data)
all_input_ids = [x for x in all_input_ids if x and len(x) > 0]
all_attention_masks = [x for x in all_attention_masks if x and len(x) > 0]
all_labels = [x for x in all_labels if x and len(x) > 0]

In [ ]:
input_ids_tensor = torch.tensor(all_input_ids)
attention_mask_tensor = torch.tensor(all_attention_masks)
labels_tensor = torch.tensor(all_labels)

In [ ]:
print(input_ids_tensor[1])
print(attention_mask_tensor[1])
print(labels_tensor[1])

In [ ]:
from sklearn.model_selection import train_test_split
tr_inputs, val_inputs, tr_tags, val_tags, tr_masks, val_masks = train_test_split(all_input_ids, all_labels, all_attention_masks, random_state=2025, test_size=0.2)

# LSTM

Sử dụng BiLSTM vì nhận diện thực thể phải dựa trên ngữ cảnh trước và sau từ đó   

LSTM truyền thống chỉ nhìn từ trái qua phải -> Mất ảnh hưởng của token đằng sau.

In [ ]:
#! pip install pytorch-crf

In [ ]:
import torch.nn as nn
from torchcrf import CRF

In [ ]:

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels):
        super(BiLSTM_CRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim // 2, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        embeddings = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embeddings)
        logits = self.hidden2tag(lstm_out)

        if labels is not None:
            loss = -self.crf(logits, labels, mask=attention_mask.byte(), reduction='mean')
            return loss
        else:
            return self.crf.decode(logits, mask=attention_mask.byte())


In [ ]:
vocab_size = tokenizer.vocab_size
embed_dim = 128
hidden_dim = 256
num_labels = len(label2id)

model = BiLSTM_CRF(vocab_size, embed_dim, hidden_dim, num_labels)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

for epoch in range(10):
    model.train()
    loss = model(input_ids_tensor, attention_mask_tensor, labels_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")


In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(input_ids_tensor, attention_mask_tensor)

In [ ]:
print(predictions[0])

In [ ]:
def decode_labels(predictions, id2label):
    return [[id2label[idx] for idx in seq] for seq in predictions]

decoded_preds = decode_labels(predictions, id2label)
decoded_labels = decode_labels(all_labels, id2label)
print(decoded_labels[0])

In [ ]:
#! pip install seqeval

In [ ]:
from seqeval.metrics import classification_report

print(classification_report(decoded_labels, decoded_preds))

# Statistical Learing approach
Thuật toán Conditional Random Fields (CRF) là thuật toán xác suất có điều kiện. Sự khác biệt ở mô hình này là sự phân phối xác suất theo điều kiện P(y/x), và xác suất này sẽ cố gắng đưa về xác suất bình thường: P(y, x).
Trong thuật toán CRFs, đầu vào là tập hợp các thuộc tính (dạng số thực) từ tập dữ liệu đầu vào theo một quy tắc. Trọng số của biểu thức với các thuộc tính đầu vào cùng các nhãn đã được găn thẻ trước đó và task sẽ được dùng để dự đoán cho việc nhãn gắn hiện tại. Ta sẽ ước lượng trọng số sao cho chỉ số likelihood của nhãn trog bộ dữ liệu train là cực đại.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

def convert_to_ner_format(row):
    text = row["content"]
    annotations = row["annotation"]

    # Tokenize câu
    tokens = word_tokenize(text)
    labels = ["O"] * len(tokens)  # Mặc định là "O" (Outside)

    for ann in annotations:
        entity_label = ann["label"][0]
        for point in ann["points"]:
            entity_text = point["text"]
            entity_start = point["start"]
            entity_end = point["end"]

            # Tokenize thực thể
            entity_tokens = word_tokenize(entity_text)

            # Xác định vị trí trong danh sách token gốc
            for i in range(len(tokens)):
                token = tokens[i]
                token_start = text.find(token)
                token_end = token_start + len(token)

                # Gán nhãn
                if token_start >= entity_start and token_end <= entity_end:
                    if i == 0:
                        labels[i] = f"B-{entity_label}"  # Beginning
                    else:
                        labels[i] = f"I-{entity_label}"  # Inside

    return list(zip(tokens, labels))

df["ner_data"] = df.apply(convert_to_ner_format, axis=1)

# Kiểm tra kết quả
for tokens_labels in df["ner_data"]:
    print(tokens_labels)
